<div align="center">

### **PARCIAL #2 SEÑALES Y SISTEMAS - 2025 2S**
#### **Estudiante:** Martín Ramírez Espinosa
##### Departamento de Ingeniería Eléctrica, Electrónica y Computación
##### Universidad Nacional de Colombia - Sede Manizales

</div>

---

---

**1.** Sea el demodulador AM presentado en la siguiente Figura:

![Demodulation system diagram](demodulation-system.png)

Asumiendo $\theta_0 = 0$, determine el espectro de Fourier (teórico) en cada etapa del sistema. Luego, utilizando 5 segundos de una canción de YouTube como mensaje $m(t)$, grafique cada una de las etapas principales (tiempo y frecuencia) del proceso de Modulación y Demodulación del mensaje y reproduzca el audio correspondiente. Para la etapa de filtrado pasa bajas utilice un filtrado ideal implementado en el dominio de la frecuencia mediante FFT.

---

**2.** Encuentre la función de transferencia que caracteriza el sistema masa-resorte-armotiguador presentado en la siguiente Figura:

![Spring mass damper diagram](spring-mass-damper.png)

Posteriormente, encuentre el sistema equivalente del modelo anterior a partir del circuito eléctrico:

![RLC system diagram](rlc-system.png)

Finalmente, proponga unos valores de $m, k$ y $c$ y sus equivalentes $R, L$ y $C$ para simular un sistema subamortiguado, sobreamortiiguado y de amortiguamiento crítico; también determine el factor de amortiguamiento, la frecuencia natural amortiguada, la frecuencia natural no amortiguada, el tiempo pico, el tiempo de levantamiento y el tiempo de establecimiento en cada caso.

Para cada caso, grafique el diagrama de polos y ceros, el diagrama de Bode, la respuesta impuslo, respuesta escalón y respuesta rampa.

Repita el proceso para modo lazo cerrado.